In [1]:
import pandas as pd
import numpy as np
import tqdm
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
intents_tweetset = pd.read_excel("/content/drive/My Drive/preproc_data/training_data/intents_tweetset.xlsx",sheet_name='Sheet1', header=0)
train = pd.read_excel("/content/drive/My Drive/preproc_data/training_data/train.xlsx",sheet_name='Sheet1', header=0)
dev = pd.read_excel("/content/drive/My Drive/preproc_data/training_data/dev.xlsx",sheet_name='Sheet1', header=0)
test = pd.read_excel("/content/drive/My Drive/preproc_data/training_data/newtest.xlsx",sheet_name='Sheet1', header=0)

In [4]:
data = pd.read_csv('/content/drive/My Drive/data.csv', header=0)

In [5]:
data.head()

,id,date_year,date_month,date_day,place_id,place_name,place_full_name,place_country,place_country_code,user_id,user_name,user_screen_name,user_description,lang,tweet
0,1215625674376937473,2020,1,10,NaN,NaN,NaN,NaN,NaN,733491445798162432,Tom Sandora,tomsandora,"Hospital Epidemiologist @BostonChildrens, pedi...",en,CDC health advisory about the Chinese coronavi...
1,1220167656805994496,2020,1,23,NaN,NaN,NaN,NaN,NaN,3047925973,"Ian Weissman, DO",DrIanWeissman,Radiologist @DeptVetAffairs. Chair @RadiologyA...,en,Coronavirus outbreak: doctors use robot to tre...
2,1220393385888010240,2020,1,23,NaN,NaN,NaN,NaN,NaN,891364509792833536,KingsportVijay,VRSINGHMD,"Civil Surgeon, US Dept Of Homeland Security. N...",en,Snakes Are The Likely Source of China's Deadly...
3,1220408120461201408,2020,1,23,NaN,NaN,NaN,NaN,NaN,1473069499,Natasha Azzopardi M,natasha_azzmus,Medical doctor - Director of Country Health Po...,en,The WHO Representative dealing with #coronavir...
4,1220444099200921601,2020,1,23,NaN,NaN,NaN,NaN,NaN,905276701,Wondimu Yirga,WondeYirga,"Public health specialist (RN, BSN,MPH), Regist...",en,CDC confirms first US case of coronavirus that...


In [6]:
def isNaN(string):
    return string != string

def strip_tweets(tweet):
    if not isNaN(tweet):
        tweet = tweet.replace('\n',' ')
    return tweet

data['stripped'] = data['tweet'].apply(lambda x: strip_tweets(x))

data_tweets = data['stripped'].values

In [7]:
train.head()

,Unnamed: 0,index,tweet,topic,intents_intersection,intents_union,intents_full_match
0,0,489,Shutdown orders prevented about 60 million nov...,"{'ECONOMY AND SOCIETY: CHINA: China (Wuhan)', ...",{'PROVIDE INFORMATION: PROVIDE INFORMATION PRE...,{'PROVIDE INFORMATION: PROVIDE INFORMATION PRE...,False
1,1,485,Britain's government is exploiting fear to sui...,{'ECONOMY AND SOCIETY: GOVERNMENT RESPONSE: ho...,{'CRITICIZE: CRITICIZE GOVERNMENT: criticizes ...,{'CRITICIZE: CRITICIZE GOVERNMENT: criticizes ...,False
2,2,2272,@JAMA_current @NIAIDNews @NEJM It looks like t...,{'SYMPTOMS AND SPREAD: INFECTION AND TRANSMISS...,{'PROVIDE INFORMATION: PROVIDE INFORMATION INF...,{'PROVIDE INFORMATION: PROVIDE INFORMATION INF...,True
3,3,5996,Just got this amazing thank you from @RedCross...,"{""HEALTHCARE: HEALTH ORGANIZATION: health orga...",{'PROVIDE INFORMATION: PROVIDE INFORMATION OTH...,{'PROVIDE INFORMATION: PROVIDE INFORMATION OTH...,True
4,4,333,Coronavirus pressures 'put organ transplants a...,{'SYMPTOMS AND SPREAD: HEALTH DISPARITIES: dis...,set(),{'PROVIDE INFORMATION: PROVIDE INFORMATION OTH...,False


In [8]:
import json

with open('/content/drive/My Drive/preproc_data/training_data/idx2topicset.json') as json_file:
    idx2topicset = json.load(json_file)

with open('/content/drive/My Drive/preproc_data/training_data/idx2intentset.json') as json_file:
    idx2intentset = json.load(json_file)

# Intent - separating into train/test/dev

In [9]:
intents_train_indices = train['index'].values
intents_dev_indices = dev['index'].values
intents_test_indices = test['index'].values

intents_train_intents = []
for i, index in enumerate(intents_train_indices):
    intents_train_intents.append(idx2intentset[str(int(index))])

intents_dev_intents = []
for i, index in enumerate(intents_dev_indices):
    intents_dev_intents.append(idx2intentset[str(int(index))])

intents_test_intents = []
for i, index in enumerate(intents_test_indices):
    intents_test_intents.append(idx2intentset[str(int(index))])

intents_train = pd.DataFrame()
intents_train['index'] = intents_train_indices
intents_train['tweet'] = train['tweet'].values
intents_train['topic'] = train['topic'].values
intents_train['intent'] = np.array(intents_train_intents)

intents_dev = pd.DataFrame()
intents_dev['index'] = intents_dev_indices
intents_dev['tweet'] = dev['tweet'].values
intents_dev['topic'] = dev['topic'].values
intents_dev['intent'] = np.array(intents_dev_intents)

intents_test = pd.DataFrame()
intents_test['index'] = intents_test_indices
intents_test['tweet'] = test['tweet'].values
intents_test['topic'] = test['topic'].values
intents_test['intent'] = np.array(intents_test_intents)

<ipython-input-9-fbb029c471ff>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  intents_train['intent'] = np.array(intents_train_intents)
<ipython-input-9-fbb029c471ff>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  intents_dev['intent'] = np.array(intents_dev_intents)
<ipython-input-9-fbb029c471ff>:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when cre

In [ ]:
reset_selective -f data_tweets

### Looking at distibution of intents in train/dev/test sets:

In [10]:
def extract_topic_intent(string):
    words = string.split(':')
    return words[0]

# subintentdict = {}

# for intentlist in intents_train_intents:
#     for element in intentlist:
#         if element in subintentdict:
#             subintentdict[element] += 1
#         else:
#             subintentdict[element] = 1

# intentdict = {}

# for intentlist in intents_train_intents:
#     for element in intentlist:
#         if extract_topic_intent(element) in intentdict:
#             intentdict[extract_topic_intent(element)] += 1
#         else:
#             intentdict[extract_topic_intent(element)] = 1

# keys = intentdict.keys()
# values = intentdict.values()
# intentdict = pd.DataFrame()
# intentdict['topic'] = keys
# intentdict['frequency'] = values
# intentdict.to_csv('intent_dict_train.csv')

# keys = subintentdict.keys()
# values = subintentdict.values()
# subintentdict = pd.DataFrame()
# subintentdict['topic'] = keys
# subintentdict['frequency'] = values
# subintentdict.to_csv('subintent_dict_train.csv')

In [ ]:
# subintentdict = {}

# for intentlist in intents_dev_intents:
#     for element in intentlist:
#         if element in subintentdict:
#             subintentdict[element] += 1
#         else:
#             subintentdict[element] = 1

# intentdict = {}

# for intentlist in intents_dev_intents:
#     for element in intentlist:
#         if extract_topic_intent(element) in intentdict:
#             intentdict[extract_topic_intent(element)] += 1
#         else:
#             intentdict[extract_topic_intent(element)] = 1

# keys = intentdict.keys()
# values = intentdict.values()
# intentdict = pd.DataFrame()
# intentdict['topic'] = keys
# intentdict['frequency'] = values
# intentdict.to_csv('intent_dict_dev.csv')

# keys = subintentdict.keys()
# values = subintentdict.values()
# subintentdict = pd.DataFrame()
# subintentdict['topic'] = keys
# subintentdict['frequency'] = values
# subintentdict.to_csv('subintent_dict_dev.csv')

In [ ]:
# subintentdict = {}

# for intentlist in intents_test_intents:
#     for element in intentlist:
#         if element in subintentdict:
#             subintentdict[element] += 1
#         else:
#             subintentdict[element] = 1

# intentdict = {}

# for intentlist in intents_test_intents:
#     for element in intentlist:
#         if extract_topic_intent(element) in intentdict:
#             intentdict[extract_topic_intent(element)] += 1
#         else:
#             intentdict[extract_topic_intent(element)] = 1

# keys = intentdict.keys()
# values = intentdict.values()
# intentdict = pd.DataFrame()
# intentdict['topic'] = keys
# intentdict['frequency'] = values
# intentdict.to_csv('intent_dict_test.csv')

# keys = subintentdict.keys()
# values = subintentdict.values()
# subintentdict = pd.DataFrame()
# subintentdict['topic'] = keys
# subintentdict['frequency'] = values
# subintentdict.to_csv('subintent_dict_test.csv')

# Preprocessing

### Imports

In [11]:
!pip install transformers
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer

from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainingArguments, pipeline)

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 109.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


### Ekphrasis Normalizing

In [ ]:
# !pip install ekphrasis
# from ekphrasis.classes.preprocessor import TextPreProcessor
# from ekphrasis.classes.tokenizer import SocialTokenizer
# from ekphrasis.dicts.emoticons import emoticons
# from ekphrasis.classes.segmenter import Segmenter
# from ekphrasis.classes.spellcorrect import SpellCorrector

# def preprocessing(tweets):
#     text_processor = TextPreProcessor(
#         # terms that will be normalized
#         normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
#             'time', 'url', 'date', 'number'],
#         # terms that will be annotated
#         annotate={"hashtag", "allcaps", "elongated", "repeated",
#             'emphasis', 'censored'},
#         fix_html=True,  # fix HTML tokens

#         # corpus from which the word statistics are going to be used
#         # for word segmentation
#         segmenter="twitter",

#         # corpus from which the word statistics are going to be used
#         # for spell correction
#         corrector="twitter",

#         unpack_hashtags=True,  # perform word segmentation on hashtags
#         unpack_contractions=True,  # Unpack contractions (can't -> can not)
#         spell_correct_elong=False,  # spell correction for elongated words

#         # select a tokenizer. You can use SocialTokenizer, or pass your own
#         # the tokenizer, should take as input a string and return a list of tokens
#         tokenizer=SocialTokenizer(lowercase=True).tokenize,

#         # list of dictionaries, for replacing tokens extracted from the text,
#         # with other expressions. You can pass more than one dictionaries.
#         dicts=[emoticons]
#     )

#     processed = []
#     for tweet in tweets:
#         processed.append(" ".join(text_processor.pre_process_doc(tweet)))

#     return processed

# tweets = train['tweet'].values

# processed_tweets = preprocessing(tweets)


### BERTweet Normalizing

In [12]:
!pip install emoji
from emoji import demojize
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token


def normalizeTweet(tweet, tweet_tokenizer):
    tokens = tweet_tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = (
        normTweet.replace("cannot ", "can not ")
        .replace("n't ", " n't ")
        .replace("n 't ", " n't ")
        .replace("ca n't", "can't")
        .replace("ai n't", "ain't")
    )
    normTweet = (
        normTweet.replace("'m ", " 'm ")
        .replace("'re ", " 're ")
        .replace("'s ", " 's ")
        .replace("'ll ", " 'll ")
        .replace("'d ", " 'd ")
        .replace("'ve ", " 've ")
    )
    normTweet = (
        normTweet.replace(" p . m .", "  p.m.")
        .replace(" p . m ", " p.m ")
        .replace(" a . m .", " a.m.")
        .replace(" a . m ", " a.m ")
    )

    return " ".join(normTweet.split())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 4.4 MB/s eta 0:00:00


### GET DATA & LABELS

In [13]:
# GET TENSORS
def get_tensors(tweets):

    tweet_tokenizer = TweetTokenizer()
    input_ids = []
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-covid19-base-cased")

    for tweet in tqdm.tqdm(tweets):
        normalized_tweet = normalizeTweet(tweet, tweet_tokenizer)
        input_ids.append(normalized_tweet)

    torch_tweets = tokenizer.batch_encode_plus(batch_text_or_text_pairs=input_ids, is_split_into_words=True, padding='max_length', truncation=True, max_length=107)

#     list_of_inputs = torch_tweets['input_ids']
#     tensors = []

#     for record in list_of_inputs:
#         tensors.append(torch.tensor(record))

#     tensors = torch.stack(tensors)

    return torch_tweets

# ONLY FOR MAIN TOPICS #
def get_main_topics(idx2topicset):
    idx2maintopics = {}

    for key, value in idx2topicset.items():
        maintopics = set()
        for topic in value:
            maintopics.add(extract_topic_intent(topic))

        idx2maintopics[key] = list(maintopics)

    return idx2maintopics

# GET MLB
def get_mlb(idx2topic):
    all_topics = list(idx2topic.values())
    all_indices = list(idx2topic.keys())

    mlb = MultiLabelBinarizer()
    allbinarizedlabels = mlb.fit_transform(all_topics)

    idx2binarizedlabel = {}
    for i, key in enumerate(all_indices):
        idx2binarizedlabel[key] = allbinarizedlabels[i]

    return mlb, idx2binarizedlabel

def get_mlb(idx2intent):
    all_intents = list(idx2intent.values())
    all_indices = list(idx2intent.keys())

    mlb = MultiLabelBinarizer()
    allbinarizedlabels = mlb.fit_transform(all_intents)

    idx2binarizedlabel = {}
    for i, key in enumerate(all_indices):
        idx2binarizedlabel[key] = allbinarizedlabels[i]

    return mlb, idx2binarizedlabel

def get_joint_labels(index_values, idx2binarizedtopiclabel, idx2binarizedintentlabel):
  labels = []
  for index in index_values:
      labels.append(torch.cat((torch.tensor(idx2binarizedtopiclabel[str(int(index))]), torch.tensor(idx2binarizedintentlabel[str(int(index))]))))

  labels = torch.stack(labels)

  return labels

# GET LABELS
def get_labels(index_values, idx2binarizedlabel):

    labels = []
    for index in index_values:
        labels.append(torch.tensor(idx2binarizedlabel[str(int(index))]))

    labels = torch.stack(labels)

    return labels


In [14]:
# GET MAIN TOPIC DICT

idx2maintopics = get_main_topics(idx2topicset)
idx2mainintents = get_main_topics(idx2intentset)

In [15]:
# GET MULTI LABEL BINARIZER (MLB)

topic_mlb, idx2binarizedtopiclabel = get_mlb(idx2maintopics)
intent_mlb, idx2binarizedintentlabel = get_mlb(idx2mainintents)

In [16]:
# DECIDE WHAT WE ARE RUNNING

joint = True
intent_bool = False
topic_bool = False

In [17]:
alltrain = pd.concat([train, dev, test])

alltrain_tensors = get_tensors(alltrain['tweet'].values)
alltrain_labels = None

if intent_bool == True:
  alltrain_labels = get_joint_labels(alltrain['index'].values, idx2binarizedintentlabel)

if topic_bool == True:
  alltrain_labels = get_joint_labels(alltrain['index'].values, idx2binarizedtopiclabel)

if joint == True:
  alltrain_labels = get_joint_labels(alltrain['index'].values, idx2binarizedtopiclabel, idx2binarizedintentlabel)

# PREPARING TRAINING DATA

# train_tensors = get_tensors(train['tweet'].values)

# train_labels = None

# if intent_bool == True:
#   train_labels = get_joint_labels(train['index'].values, idx2binarizedintentlabel)

# if topic_bool == True:
#   train_labels = get_joint_labels(train['index'].values, idx2binarizedtopiclabel)

# if joint == True:
#   train_labels = get_joint_labels(train['index'].values, idx2binarizedtopiclabel, idx2binarizedintentlabel)


# PREPARING DEV DATA

# dev_tensors = get_tensors(dev['tweet'].values)

# dev_labels = None

# if intent_bool == True:
#   dev_labels = get_joint_labels(dev['index'].values, idx2binarizedintentlabel)

# if topic_bool == True:
#   dev_labels = get_joint_labels(dev['index'].values, idx2binarizedtopiclabel)

# if joint == True:
#   dev_labels = get_joint_labels(dev['index'].values, idx2binarizedtopiclabel, idx2binarizedintentlabel)


# PREPARING TEST DATA

test_tensors = get_tensors(test['tweet'].values)

test_labels = None

if intent_bool == True:
  test_labels = get_joint_labels(test['index'].values, idx2binarizedintentlabel)

if topic_bool == True:
  test_labels = get_joint_labels(test['index'].values, idx2binarizedtopiclabel)

if joint == True:
  test_labels = get_joint_labels(test['index'].values, idx2binarizedtopiclabel, idx2binarizedintentlabel)

100%|██████████| 932/932 [00:00<00:00, 1428.24it/s]


In [18]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [19]:
class TwitterCovidData(Dataset):
    def __init__(self, tensors, labels):
        self.data = tensors
        self.labels = labels.to(torch.float32).to(device)

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, index):
        inputs = {k: torch.tensor(v[index]).to(device) for k, v in self.data.items()}
        inputs["labels"] =  self.labels[index]
        return inputs

class TwitterCovidDataPredict(Dataset):
    def __init__(self, tensors):
        self.data = tensors

    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, index):
        inputs = {k: torch.tensor(v[index]).to(device) for k, v in self.data.items()}
        return inputs

JOINT INTENTS + TOPICS

In [20]:
# train_data = TwitterCovidData(train_tensors, train_labels)

# dev_data = TwitterCovidData(dev_tensors, dev_labels)

test_data = TwitterCovidData(test_tensors, test_labels)

# alltrain_data = TwitterCovidData(alltrain_tensors, alltrain_labels)

### COMPUTE METRICS

In [21]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro': f1_macro_average,
               'roc_auc_micro': roc_auc,
               'accuracy': accuracy}
    return metrics


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

def compute_joint_metrics(p: EvalPrediction):
  preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions

  threshold=0.5

  predictions = preds
  labels = p.label_ids

  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))
  # next, use threshold to turn them into integer predictions
  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs >= threshold)] = 1
  # finally, compute metrics
  y_true = labels

  y_true_topic = labels[:, :9]
  y_true_intent = labels[:, 9:]

  y_pred_topic = y_pred[:, :9]
  y_pred_intent = y_pred[:, 9:]

  avg_f1_micro = None
  avg_f1_macro = None
  roc_auc = None
  accuracy = None

  # double check dimnesions of labels/logits
  # split into two pairs of matrices topic preds topic true, intents preds, intent preds, and then report both spearately
  f1_topics_micro = None
  f1_intents_micro = None
  f1_topics_macro = None
  f1_intents_macro = None

  if torch.sum(torch.from_numpy(y_true_intent)) == 0:
    f1_macro_average = f1_score(y_true=y_true_topic, y_pred=y_pred_topic, average='macro')
    f1_micro_average = f1_score(y_true=y_true_topic, y_pred=y_pred_topic, average='micro')
    f1_topics_micro = f1_micro_average
    f1_topics_macro = f1_macro_average
    avg_f1_micro = f1_micro_average
    avg_f1_macro = f1_macro_average
    roc_auc = roc_auc_score(y_true_topic, y_pred_topic, average = 'micro')
    accuracy = accuracy_score(y_true_topic, y_pred_topic)
  else:
    f1_topics_micro = f1_score(y_true=y_true_topic, y_pred=y_pred_topic, average='micro')
    f1_topics_macro = f1_score(y_true=y_true_topic, y_pred=y_pred_topic, average='macro')
    f1_intents_micro = f1_score(y_true=y_true_intent, y_pred=y_pred_intent, average='micro')
    f1_intents_macro = f1_score(y_true=y_true_intent, y_pred=y_pred_intent, average='macro')
    avg_f1_micro = (f1_topics_micro + f1_intents_micro)/2
    avg_f1_macro = (f1_topics_macro + f1_intents_macro)/2
    roc_auc = (roc_auc_score(y_true_topic, y_pred_topic, average = 'micro') + roc_auc_score(y_true_intent, y_pred_intent, average = 'micro'))/2
    accuracy = (accuracy_score(y_true_topic, y_pred_topic) + accuracy_score(y_true_intent, y_pred_intent))/2

  # return as dictionary
  metrics = {'f1_macro': avg_f1_macro,
             'f1_micro': avg_f1_micro,
             'f1_topics_macro': f1_topics_macro,
             'f1_topics_micro': f1_topics_micro,
              'f1_intents_macro': f1_intents_macro,
             'f1_intents_micro': f1_intents_micro,
              'roc_auc_micro': roc_auc,
              'accuracy': accuracy}

  return metrics


### TRAINING AND EVALUATION

Initializing Functions and Training **Loop**

In [22]:
classes = None

if intent_bool == True:
  classes = list(intent_mlb.classes_)

if topic_bool == True:
  classes = list(topic_mlb.classes_)

if joint == True:
  classes = list(topic_mlb.classes_) + list(intent_mlb.classes_)

id2label = {idx:label for idx, label in enumerate(classes)}
label2id = {label:idx for idx, label in enumerate(classes)}

id2label

{0: 'CAUSE',
 1: 'ECONOMY AND SOCIETY',
 2: 'HEALTHCARE',
 3: 'MISINFORMATION',
 4: 'OTHER',
 5: 'PREVENTION',
 6: 'RESOURCE',
 7: 'SYMPTOMS AND SPREAD',
 8: 'TREATMENT',
 9: 'CRITICIZE',
 10: 'HONOR',
 11: 'PRAISE',
 12: 'PREDICT',
 13: 'PROMOTE OWN WORK',
 14: 'PROVIDE INFORMATION',
 15: 'RECOMMEND',
 16: 'SEEK INFORMATION',
 17: 'SHARE STORY'}

In [23]:
from sklearn.metrics import classification_report

def get_hyperparams(lr, tbs, ebs, epochs, wd):
  hyperparams = {}
  hyperparams['learning_rate'] = lr
  hyperparams['train_batch_size'] = tbs
  hyperparams['eval_batch_size'] = ebs
  hyperparams['num_train_epochs'] = epochs
  hyperparams['weight_decay'] = wd

  return hyperparams

def generate_results(predictions, labels, classes, hyperparams, filenum, threshold=0.5):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))
  # next, use threshold to turn them into integer predictions
  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs >= threshold)] = 1

  y_true = labels
  report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

  metrics = multi_label_metrics(predictions, labels, threshold=0.5)

  results_Df = pd.DataFrame.from_dict(report, orient='index').reset_index()
  results_Df.rename(columns={'index':'label'}, inplace=True)
  results_Df.loc[len(results_Df.index)] = ['roc_auc_micro', metrics['roc_auc_micro'], '--', '--', '--']
  results_Df.loc[len(results_Df.index)] = ['accuracy', metrics['accuracy'], '--', '--', '--']

  results_Df['learning_rate'] = hyperparams['learning_rate']
  results_Df['train_batch_size'] = hyperparams['train_batch_size']
  results_Df['eval_batch_size'] = hyperparams['eval_batch_size']
  results_Df['num_train_epochs'] = hyperparams['num_train_epochs']
  results_Df['weight_decay'] = hyperparams['weight_decay']
  results_Df.set_index(['num_train_epochs', 'learning_rate','train_batch_size', 'eval_batch_size', 'weight_decay', 'label'], inplace=True)

  filename = 'results_' + str(filenum) + '.csv'
  results_Df.to_csv(filename)

  return results_Df

In [ ]:
# from transformers import Trainer
# class MyTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):

#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs[0]

#         loss = None
#         criterion = torch.nn.BCEWithLogitsLoss()

#         labels_topic = labels[:, :9]
#         labels_intent = labels[:, 9:]

#         logits_topic = logits[:, :9]
#         logits_intent = logits[:, 9:]

#         if torch.sum(labels_intent) == 0:
#           loss = criterion(logits_topic, labels_topic)
#         else:
#           loss = (criterion(logits_topic, labels_topic) + criterion(logits_intent, labels_intent))/2

#         return (loss, outputs) if return_outputs else loss

Train **Loop**

In [ ]:
# lr = 4e-5
# tbs = 32
# ebs = 32
# epochs = 15
# wd = 0.1

# hyperparams = get_hyperparams(lr, tbs, ebs, epochs, wd)

# model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-covid19-base-cased", num_labels=alltrain_data.labels.shape[1], problem_type='multi_label_classification', id2label=id2label, label2id=label2id).to(device)

# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-covid19-base-cased")

# training_args = TrainingArguments(
#     output_dir='output',
#     learning_rate=lr,
#     per_device_train_batch_size=tbs,
#     per_device_eval_batch_size=ebs,
#     num_train_epochs=epochs,
#     weight_decay=wd,
#     metric_for_best_model="f1_macro",
#     do_train=True,
#     do_eval=False,
#     evaluation_strategy="no",
#     save_strategy="no",
#     load_best_model_at_end=True,
#     save_total_limit=3,

# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=alltrain_data,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

# if joint == True:
#   trainer = MyTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=alltrain_data,
#     tokenizer=tokenizer,
#     compute_metrics=compute_joint_metrics,
#   )

# metrics = trainer.train()

In [ ]:
# trainer.save_model('/content/drive/My Drive/preproc_data/saved_model')

**Predict**

In [ ]:
alldata_tensors = get_tensors(data_tweets)
# alldata = TwitterCovidDataPredict(alldata_tensors)

  3%|▎         | 39829/1255720 [00:10<07:30, 2696.23it/s]

In [ ]:
# input_ids = alldata_tensors['input_ids']
# attention_mask = alldata_tensors['attention_mask']
# token_type_ids = alldata_tensors['token_type_ids']

# input_ids = torch.tensor(input_ids)

In [24]:
input_ids = test_tensors['input_ids']
attention_mask = test_tensors['attention_mask']
token_type_ids = test_tensors['token_type_ids']

input_ids = torch.tensor(input_ids)

In [27]:
attention_mask = torch.tensor(attention_mask)

In [ ]:
torch.save(input_ids, '/content/drive/My Drive/preproc_data/input_ids.pt')

In [ ]:
torch.save(attention_mask, '/content/drive/My Drive/preproc_data/attention_mask.pt')

In [ ]:
token_type_ids = torch.tensor(token_type_ids)

In [ ]:
torch.save(token_type_ids,'/content/drive/My Drive/preproc_data/token_type_ids.pt')

In [25]:
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/My Drive/preproc_data/saved_model')
# outputs = model(input_ids, attention_mask)

In [ ]:
outputs = model(input_ids, attention_mask)

In [ ]:
# predictions = trainer.predict(alldata)

In [ ]:
# results = generate_results(predictions.predictions, test_labels, classes, hyperparams, 2, threshold=0.5)

In [ ]:
# results

In [ ]:
# results

In [ ]:
# results.to_csv('results_joint.csv')

In [ ]:
# results1

In [ ]:
# results2

Display **Predictions**

In [ ]:
# preds = pd.DataFrame()

# topics = test['topic'].values

# preds['index'] = test['index'].values
# preds['tweet'] = test['tweet'].values


# maintops = []
# for id in test['index'].values:

#   topiclist = idx2maintopics[str(int(id))]
#   maintops.append(topiclist)

# preds['topic'] = np.array(maintops)

# multi_label_metrics(predictions.predictions, test_labels, threshold=0.5)

In [ ]:
# sigmoid = torch.nn.Sigmoid()
# probs = sigmoid(torch.Tensor(predictions.predictions))
# # next, use threshold to turn them into integer predictions
# y_pred = np.zeros(probs.shape)
# y_pred[np.where(probs >= 0.5)] = 1

In [ ]:
# mlb.classes_

In [ ]:
# pred_topics = []

# for pred in y_pred:
#   each_pred = []
#   pred_list = list(pred)

#   for i, p in enumerate(pred_list):
#     if p == 1:
#       each_pred.append(mlb.classes_[i])

#   pred_topics.append(each_pred)

# pred_topics_array = np.array(pred_topics)
# preds['predictions'] = pred_topics_array

In [ ]:
# preds.head()

In [ ]:
# intersections = []

# num_correct_per_topic = {}
# topics = preds['topic'].values

# for i, id in enumerate(test['index'].values):
#   topicset = set(topics[i])
#   preds_set = set(pred_topics[i])

#   inter = topicset.intersection(preds_set)

#   for element in inter:
#     if element in num_correct_per_topic:
#       num_correct_per_topic[element] += 1
#     else:
#       num_correct_per_topic[element] = 0

#   intersections.append(inter)

# preds['intersection'] = np.array(intersections)
# # num_correct_df = pd.DataFrame.from_dict(num_correct_per_topic)
# # num_correct_df.to_excel('num_correct_per_topic.xlsx')
# preds.to_excel('test_preds.xlsx')

In [ ]:
# preds.head()

In [ ]:
# num_correct_per_topic

In [ ]:
# preds.to_excel('test_preds.xlsx')